In [1]:
# As single_peakon, but trained on multipeakons!

In [2]:
%run base/base_eq.ipynb
%run base/base_plot.ipynb
%run base/base_ml.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
def gen_one_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a single-peakon solution on a coarse grid, and the labels are the values of
    the single-peakon solution on a fine grid.
    """
    point, peakon = gen_one_peakon()
    return sol_on_grid(point, peakon)

def gen_two_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a two-peakon solution on a coarse grid, and the labels are the values of
    the two-peakon solution on a fine grid.
    """
    point, twopeakon = gen_two_peakon()
    return sol_on_grid(point, twopeakon)

def gen_one_data():
    return tools.random_function(gen_one_peakon_on_grid, gen_two_peakon_on_grid)

In [6]:
### DNN hyperparameters
hidden_units = [600] * 10
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.0
drop_type = 'dropout'
preprocessor = ScaleDataOverall(momentum=0.999)
activation = tf.nn.relu
uuid = None

### Training hyperparameters #1
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/test/'
model_dir = model_dir_str(model_dir_base, hidden_units, logits, 
                          drop_rate=drop_rate, drop_type=drop_type,
                          preprocessor=preprocessor, activation=activation, 
                          uuid=uuid)

dnn = create_dnn(hidden_units, logits, drop_rate=drop_rate,
                 drop_type=drop_type, model_dir=model_dir, 
                 log_steps=log_steps, activation=activation, 
                 gradient_clip=gradient_clip, batch_norm=False)
preprocessor.load(model_dir)

if TRAIN:
    ### Training hyperparameters #2
    batch_size = 30
    steps = 20000
    train_input_fn = preprocessor.data(BatchData.from_func(gen_one_data=gen_one_data, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn, max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

    batch_size = 300
    steps = 30000
    train_input_fn = preprocessor.data(BatchData.from_func(gen_one_data=gen_one_data, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn, max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

INFO:tensorflow:Using config: {'_model_dir': './saved_models/test/600x10_121_D00_ScaleDataOverall_relu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f4dbdd8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:No processor checkpoint file './saved_models/test/600x10_121_D00_ScaleDataOverall_relu/processor-checkpoint' found.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring param

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`